In [12]:
import paho.mqtt.client as mqtt
from pymongo import MongoClient
import json
import datetime
import numpy as np
from scipy.stats import skew, kurtosis
from biosppy.signals import ecg
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Charger le modèle XGBoost
bst = xgb.Booster()
bst.load_model('C:/Users/dell/OneDrive/Bureau/mastere2/projet_fin_etude/page web/page daccueil/infermier/ecg_classification.xgb')  # Assurez-vous que le chemin vers le modèle est correct

# Configuration MongoDB
mongo_client = MongoClient('mongodb://localhost:27017/')

# Configuration MQTT
mqtt_broker = "192.168.1.35"  # L'adresse IP de votre broker MQTT
mqtt_port = 1883

# Tableau temporaire pour stocker les valeurs ECG
ecg_values = []

# Callback pour la connexion
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT broker")
    else:
        print(f"Failed to connect to MQTT broker with result code {rc}")

    # S'abonner uniquement au topic ECG avec QoS 2
    client.subscribe("vital_signs/+/ecg", qos=2)

# Fonction pour extraire les caractéristiques du signal ECG
def extract_features(signal):
    features = []
    features.append(np.mean(signal))
    features.append(np.std(signal))
    features.append(np.min(signal))
    features.append(np.max(signal))
    features.append(skew(signal))
    features.append(kurtosis(signal))
    
    rpeaks = ecg.ecg(signal=signal, sampling_rate=360, show=False)['rpeaks']
    
    rr_intervals = np.diff(rpeaks) / 360.0  # Intervalles R-R en secondes
    heart_rate = 60 / rr_intervals  # Fréquence cardiaque moyenne
    
    features.append(np.mean(rr_intervals))
    features.append(np.std(rr_intervals))
    features.append(np.min(rr_intervals))
    features.append(np.max(rr_intervals))
    features.append(skew(rr_intervals))
    features.append(kurtosis(rr_intervals))
    
    features.append(np.mean(heart_rate))
    features.append(np.std(heart_rate))
    features.append(np.min(heart_rate))
    features.append(np.max(heart_rate))
    
    return np.array(features)

# Callback pour la réception de messages
def on_message(client, userdata, msg):
    print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")

    # Analyser la topic pour extraire le numéro de lit
    topic_parts = msg.topic.split('/')
    db_name = "hopital"
    bed_number_str = topic_parts[1]  # e.g., 'bed1'
    collection_name = topic_parts[2]  # e.g., 'ecg'

    if collection_name == "ecg":
        bed_number = int(bed_number_str.replace('bed', ''))

        try:
            ecg_data = json.loads(msg.payload.decode())
            if isinstance(ecg_data, list):
                ecg_values.extend(ecg_data)
                taille_du_tableau = len(ecg_values)
                print(f"La taille du tableau est : {taille_du_tableau}")

                if len(ecg_values) >= 3600:
                    ecg_signal = np.array(ecg_values[:3600])
                    
                    scaler = StandardScaler()
                    ecg_signal_standardized = scaler.fit_transform(ecg_signal.reshape(-1, 1)).flatten()
                    
                    signal_features = extract_features(ecg_signal_standardized)
                    
                    dtest = xgb.DMatrix(signal_features.reshape(1, -1))
                    predicted_class = int(bst.predict(dtest)[0])
                    
                    data = {
                        "bed": bed_number,
                        "values": ecg_values[:3600],
                        "valeur_predite": predicted_class,
                        "timestamp": datetime.datetime.now()
                    }

                    db = mongo_client[db_name]
                    collection = db[collection_name]
                    try:
                        collection.insert_one(data)
                        print(f"Data inserted into collection '{collection_name}' in database '{db_name}'")
                    except Exception as e:
                        print(f"Error inserting data into MongoDB: {e}")

                    ecg_values[:] = ecg_values[3600:]
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON message: {e}")

# Configuration du client MQTT
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_broker, mqtt_port, 60)

# Boucle infinie pour maintenir la connexion MQTT
client.loop_forever()


C:\Users\dell\AppData\Local\Temp\ipykernel_15120\1289317395.py:115: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Connected to MQTT broker
Received `[1981,2347,2240,1760,1427,1375,1535,1983,2351,2245,1760,1435,1387,1574,2009,2390,2131,1381,1080,1250,1687,2506,3101,2519,1370,816,933,1462,2439,3197,2640,1466,837,891,1397,2399,3184,2650,1479,830,867,1371,2386,3203,2686,1523,864,880,1377,2389,3178,2640,1488,851,884,1395,2426,3199,2624,1483,854,906,1423,2452,3187,2559,1453,857,931,1462,2482,3188,2549,1435,848,934,1472,2492,3172,2501,1392,848,967,1521,2530,3149,2448,1360,846,987,1578,2559,3127,2402,1311,848,1018,1616,2601,3104]` from `vital_signs/bed1/ecg` topic
La taille du tableau est : 100
Received `[1927,3200,3283,1919,880,672,970,1927,3216,3274,1920,890,690,978,1936,3231,3251,1892,878,689,990,1968,3258,3226,1856,870,707,1011,2013,3295,3195,1817,837,681,1004,2031,3330,3191,1805,839,688,1009,2055,3334,3146,1759,816,697,1040,2096,3375,3129,1713,800,687,1034,2106,3378,3071,1681,785,693,1051,2134,3393,3071,1663,778,689,1057,2167,3430,3056,1646,772,699,1082,2206,3454,3023,1605,752,688,1081,2210,3458,3009

KeyboardInterrupt: 

In [8]:
import paho.mqtt.client as mqtt
from pymongo import MongoClient
import json
import datetime
import joblib
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from scipy.stats import skew, kurtosis
from biosppy.signals import ecg
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import traceback
import locale

# Charger les modèles
model = joblib.load('C:/Users/dell/Downloads/prediction etat de santé.pkl')
# decision_tree_model = joblib.load('C:/Users/dell/Downloads/prediction etat de santé.pkl')
xgboost_model = xgb.Booster()
xgboost_model.load_model('C:/Users/dell/OneDrive/Bureau/mastere2/projet_fin_etude/page web/page daccueil/infermier/ecg_classification.xgb')

# Configuration MongoDB
mongo_client = MongoClient('mongodb://localhost:27017/')

# Configuration MQTT
mqtt_broker = "10.5.18.186"  # Utiliser l'adresse IP de votre broker MQTT
mqtt_port = 1883


def get_today_phone_numbers():
    # Définir la localisation en français pour obtenir le jour en français
    locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

    # Récupérer le jour d'aujourd'hui en français
    today = datetime.today().strftime('%A')
    # print(today)

    # Connecter à la base de données MongoDB
    client = MongoClient('mongodb://localhost:27017/')  # Remplacez par l'URL de connexion de votre base de données
    db = client['hopital']  # Remplacez par le nom de votre base de données
    collection = db['medecins']  # Remplacez par le nom de votre collection

    # Rechercher les numéros de téléphone des médecins dont le service correspond au jour d'aujourd'hui
    results = collection.find({'service': today}, {'_id': 0, 'phone': 1})

    # Extraire et retourner les numéros de téléphone
    phone_numbers = [result['phone'] for result in results]
    return phone_numbers

# Initialiser les dictionnaires et listes pour stocker les valeurs des signaux vitaux et ECG
vital_signs_data = {
    "temperature": None,
    "heart_rate": None,
    "spo2": None,
    "breath_rate": None
}
complete_vital_signs_data = []
ecg_values = []

# Callback pour la connexion MQTT
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT broker")
    else:
        print(f"Failed to connect to MQTT broker with result code {rc}")

    # S'abonner aux topics spécifiés
    client.subscribe("vital_signs/+/temperature", qos=1)
    client.subscribe("vital_signs/+/heart_rate", qos=1)
    client.subscribe("vital_signs/+/spo2", qos=1)
    client.subscribe("vital_signs/+/breath_rate", qos=1)
    client.subscribe("vital_signs/+/ecg", qos=2)

# Fonction pour récupérer la dernière valeur de "blood_pressure" dans MongoDB
def get_latest_blood_pressure_value():
    db = mongo_client["hopital"]
    collection = db["blood_pressure"]
    latest_document = collection.find_one(sort=[("timestamp", -1)])
    return latest_document.get("value", None) if latest_document else None

# Fonction de prédiction de l'état de santé
def predict_health_state(heart_rate, blood_pressure, oxygen_saturation, respiratory_rate, temperature):
        # Convertir les entrées en entiers
        heart_rate = float(heart_rate)
        blood_pressure = float(blood_pressure)
        oxygen_saturation = float(oxygen_saturation)
        respiratory_rate = float(respiratory_rate)
        temperature = float(temperature)

        # Exemple de données d'entrée
        data = np.array([[heart_rate, blood_pressure, oxygen_saturation, respiratory_rate, temperature]])

        # Effectuer une prédiction
        predicted_state = model.predict(data)
        health_prediction = "Normal" if predicted_state[0] == 0 else "Pas Normal"
        print("État de santé prédit:", health_prediction)

        # Afficher les règles de l'arbre de décision
        feature_names = ['heart_rate', 'blood_pressure', 'oxygen_saturation', 'respiratory_rate', 'temperature']

        # Obtenir le chemin de décision pour l'exemple de données d'entrée
        node_indicator = model.decision_path(data)
        leaf_id = model.apply(data)

        # Extraire les règles de décision
        rules = []
        for sample_id in range(len(data)):
            node_index = node_indicator.indices[node_indicator.indptr[sample_id]:node_indicator.indptr[sample_id + 1]]

            print(f"Règles pour l'échantillon {sample_id}:")
            for node_id in node_index:
                if leaf_id[sample_id] == node_id:
                    continue

                # Extraire la règle de décision
                if data[sample_id, model.tree_.feature[node_id]] <= model.tree_.threshold[node_id]:
                    threshold_sign = "<="
                else:
                    threshold_sign = ">"

                rules.append(f"Decision node {node_id}: (X[{feature_names[model.tree_.feature[node_id]]}] {threshold_sign} {model.tree_.threshold[node_id]:.2f})")

        # Afficher les règles extraites
        print("\n".join(rules))

        return health_prediction, rules

# Fonction pour extraire les caractéristiques du signal ECG
def extract_features(signal):
    features = []
    features.append(np.mean(signal))
    features.append(np.std(signal))
    features.append(np.min(signal))
    features.append(np.max(signal))
    features.append(skew(signal))
    features.append(kurtosis(signal))
    rpeaks = ecg.ecg(signal=signal, sampling_rate=360, show=False)['rpeaks']
    rr_intervals = np.diff(rpeaks) / 360.0
    heart_rate = 60 / rr_intervals
    features.append(np.mean(rr_intervals))
    features.append(np.std(rr_intervals))
    features.append(np.min(rr_intervals))
    features.append(np.max(rr_intervals))
    features.append(skew(rr_intervals))
    features.append(kurtosis(rr_intervals))
    features.append(np.mean(heart_rate))
    features.append(np.std(heart_rate))
    features.append(np.min(heart_rate))
    features.append(np.max(heart_rate))
    return np.array(features)

# Callback pour la réception de messages MQTT
def on_message(client, userdata, msg):
    print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")
    topic_parts = msg.topic.split('/')
    bed_number_str = topic_parts[1]
    signal_type = topic_parts[2]

    if signal_type in vital_signs_data:
        vital_signs_data[signal_type] = msg.payload.decode()
        if all(value is not None for value in vital_signs_data.values()):
            blood_pressure_value = get_latest_blood_pressure_value()
            bed_number = int(bed_number_str.replace('bed', ''))
            complete_data = {
                "bed": bed_number,
                "temperature": vital_signs_data["temperature"],
                "heart_rate": vital_signs_data["heart_rate"],
                "spo2": vital_signs_data["spo2"],
                "breath_rate": vital_signs_data["breath_rate"],
                "blood_pressure": blood_pressure_value,
                "timestamp": datetime.datetime.now()
            }
            complete_vital_signs_data.append(complete_data)
            try:
                val_heart_rate = float(vital_signs_data["heart_rate"])
                val_spo2 = float(vital_signs_data["spo2"])
                val_breath_rate = float(vital_signs_data["breath_rate"])
                val_temperature = float(vital_signs_data["temperature"])
                health_prediction, rules = predict_health_state(val_heart_rate, blood_pressure_value, val_spo2, val_breath_rate, val_temperature)
                db = mongo_client["hopital"]

                
                for signal, value in vital_signs_data.items():
                    collection = db[signal]
                    data = {
                        "bed": bed_number,
                        "value": value,
                        "timestamp": datetime.datetime.now()
                    }
                    collection.insert_one(data)
                    print(f"Data inserted into collection '{signal}' in database 'hopital'")
                collection = db["etat_patient"]
                patient_data = {
                    "bed": bed_number,
                    "health_prediction": health_prediction,
                    "rules": rules,
                    "timestamp": datetime.datetime.now()
                }
                collection.insert_one(patient_data)
                print("Data inserted into collection 'etat_patient' in database 'hopital'")
                for key in vital_signs_data:
                    vital_signs_data[key] = None
            except ValueError as e:
                print(f"Erreur de conversion de la valeur: {e}")
                print(traceback.format_exc())

    elif signal_type == "ecg":
        try:
            ecg_data = json.loads(msg.payload.decode())
            if isinstance(ecg_data, list):
                ecg_values.extend(ecg_data)
                if len(ecg_values) >= 3600:
                    ecg_signal = np.array(ecg_values[:3600])
                    scaler = StandardScaler()
                    ecg_signal_standardized = scaler.fit_transform(ecg_signal.reshape(-1, 1)).flatten()
                    signal_features = extract_features(ecg_signal_standardized)
                    dtest = xgb.DMatrix(signal_features.reshape(1, -1))
                    predicted_class = int(xgboost_model.predict(dtest)[0])
                    data = {
                        "bed": int(bed_number_str.replace('bed', '')),
                        "values": ecg_values[:3600],
                        "valeur_predite": predicted_class,
                        "timestamp": datetime.datetime.now()
                    }
                    db = mongo_client["hopital"]
                    collection = db["ecg"]
                    collection.insert_one(data)
                    print(f"Data inserted into collection 'ecg' in database 'hopital'")
                    ecg_values[:] = ecg_values[3600:]
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON message: {e}")

# Configuration du client MQTT
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_broker, mqtt_port, 60)

# Boucle infinie pour maintenir la connexion MQTT
client.loop_forever()


c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\dell\AppData\Local\Temp\ipykernel_15856\90667545.py:233: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


TimeoutError: timed out

In [18]:
from pymongo import MongoClient
from datetime import datetime
import locale

current_time = datetime.today().strftime('%A')
# print(current_time)

# Access date and time components
# current_date = datetime.date.today()
# print(current_date)

def get_today_phone_numbers():
    # Définir la localisation en français pour obtenir le jour en français
    locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

    # Récupérer le jour d'aujourd'hui en français
    today = datetime.today().strftime('%A')
    # print(today)

    # Connecter à la base de données MongoDB
    client = MongoClient('mongodb://localhost:27017/')  # Remplacez par l'URL de connexion de votre base de données
    db = client['hopital']  # Remplacez par le nom de votre base de données
    collection = db['medecins']  # Remplacez par le nom de votre collection

    # Rechercher les numéros de téléphone des médecins dont le service correspond au jour d'aujourd'hui
    results = collection.find({'service': today}, {'_id': 0, 'phone': 1})

    # Extraire et retourner les numéros de téléphone
    phone_numbers = [result['phone'] for result in results]
    return phone_numbers

# Exemple d'utilisation de la fonction
if __name__ == '__main__':
    phone_numbers = get_today_phone_numbers()
    for phone in phone_numbers:
        print(phone)


mercredi
0555390183


In [1]:
import paho.mqtt.client as mqtt
from pymongo import MongoClient
import json
import datetime
import locale
import joblib
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from scipy.stats import skew, kurtosis
from biosppy.signals import ecg
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import traceback
import locale
import sys
from alert import send_order
from get_phone_nember import get_today_phone_numbers

# Charger les modèles
model = joblib.load('C:/Users/dell/Downloads/prediction etat de santé.pkl')
# decision_tree_model = joblib.load('C:/Users/dell/Downloads/prediction etat de santé.pkl')
xgboost_model = xgb.Booster()
xgboost_model.load_model('C:/Users/dell/OneDrive/Bureau/mastere2/projet_fin_etude/page web/page daccueil/infermier/ecg_classification.xgb')

# Configuration MongoDB
mongo_client = MongoClient('mongodb://localhost:27017/')

# Configuration MQTT
mqtt_broker = "127.0.0.1"  # Utiliser l'adresse IP de votre broker MQTT
mqtt_port = 1883



# def get_today_phone_numbers():
#     # Définir la localisation en français pour obtenir le jour en français
#     locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

#     # Récupérer le jour d'aujourd'hui en français
#     today = datetime.today().strftime('%A')
#     # print(today)

#     # Connecter à la base de données MongoDB
#     client = MongoClient('mongodb://localhost:27017/')  # Remplacez par l'URL de connexion de votre base de données
#     db = client['hopital']  # Remplacez par le nom de votre base de données
#     collection = db['medecins']  # Remplacez par le nom de votre collection

#     # Rechercher les numéros de téléphone des médecins dont le service correspond au jour d'aujourd'hui
#     results = collection.find({'service': today}, {'_id': 0, 'phone': 1})

#     # Extraire et retourner les numéros de téléphone
#     phone_numbers = [result['phone'] for result in results]
#     return phone_numbers

# Initialiser les dictionnaires et listes pour stocker les valeurs des signaux vitaux et ECG
vital_signs_data = {
    "temperature": None,
    "heart_rate": None,
    "spo2": None,
    "breath_rate": None
}
complete_vital_signs_data = []
ecg_values = []

# Callback pour la connexion MQTT
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT broker")
    else:
        print(f"Failed to connect to MQTT broker with result code {rc}")

    # S'abonner aux topics spécifiés
    client.subscribe("vital_signs/+/temperature", qos=1)
    client.subscribe("vital_signs/+/heart_rate", qos=1)
    client.subscribe("vital_signs/+/spo2", qos=1)
    client.subscribe("vital_signs/+/breath_rate", qos=1)
    client.subscribe("vital_signs/+/ecg", qos=2)

# Fonction pour récupérer la dernière valeur de "blood_pressure" dans MongoDB
def get_latest_blood_pressure_value():
    db = mongo_client["hopital"]
    collection = db["blood_pressure"]
    latest_document = collection.find_one(sort=[("timestamp", -1)])
    return latest_document.get("value", None) if latest_document else None

# Fonction de prédiction de l'état de santé
def predict_health_state(heart_rate, blood_pressure, oxygen_saturation, respiratory_rate, temperature):
        # Convertir les entrées en entiers
        heart_rate = float(heart_rate)
        blood_pressure = float(blood_pressure)
        oxygen_saturation = float(oxygen_saturation)
        respiratory_rate = float(respiratory_rate)
        temperature = float(temperature)

        # Exemple de données d'entrée
        data = np.array([[heart_rate, blood_pressure, oxygen_saturation, respiratory_rate, temperature]])

        # Effectuer une prédiction
        predicted_state = model.predict(data)
        health_prediction = "Normal" if predicted_state[0] == 0 else "Pas Normal"
        print("État de santé prédit:", health_prediction)

        # Afficher les règles de l'arbre de décision
        feature_names = ['heart_rate', 'blood_pressure', 'oxygen_saturation', 'respiratory_rate', 'temperature']

        # Obtenir le chemin de décision pour l'exemple de données d'entrée
        node_indicator = model.decision_path(data)
        leaf_id = model.apply(data)

        # Extraire les règles de décision
        rules = []
        for sample_id in range(len(data)):
            node_index = node_indicator.indices[node_indicator.indptr[sample_id]:node_indicator.indptr[sample_id + 1]]

            print(f"Règles pour l'échantillon {sample_id}:")
            for node_id in node_index:
                if leaf_id[sample_id] == node_id:
                    continue

                # Extraire la règle de décision
                if data[sample_id, model.tree_.feature[node_id]] <= model.tree_.threshold[node_id]:
                    threshold_sign = "<="
                else:
                    threshold_sign = ">"

                rules.append(f"Decision node {node_id}: (X[{feature_names[model.tree_.feature[node_id]]}] {threshold_sign} {model.tree_.threshold[node_id]:.2f})")

        # Afficher les règles extraites
        print("\n".join(rules))

        return health_prediction, rules

# Fonction pour extraire les caractéristiques du signal ECG
def extract_features(signal):
    features = []
    features.append(np.mean(signal))
    features.append(np.std(signal))
    features.append(np.min(signal))
    features.append(np.max(signal))
    features.append(skew(signal))
    features.append(kurtosis(signal))
    rpeaks = ecg.ecg(signal=signal, sampling_rate=360, show=False)['rpeaks']
    rr_intervals = np.diff(rpeaks) / 360.0
    heart_rate = 60 / rr_intervals
    features.append(np.mean(rr_intervals))
    features.append(np.std(rr_intervals))
    features.append(np.min(rr_intervals))
    features.append(np.max(rr_intervals))
    features.append(skew(rr_intervals))
    features.append(kurtosis(rr_intervals))
    features.append(np.mean(heart_rate))
    features.append(np.std(heart_rate))
    features.append(np.min(heart_rate))
    features.append(np.max(heart_rate))
    return np.array(features)

# Callback pour la réception de messages MQTT
def on_message(client, userdata, msg):
    print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")
    topic_parts = msg.topic.split('/')
    bed_number_str = topic_parts[1]
    signal_type = topic_parts[2]

    if signal_type in vital_signs_data:
        vital_signs_data[signal_type] = msg.payload.decode()
        if all(value is not None for value in vital_signs_data.values()):
            blood_pressure_value = get_latest_blood_pressure_value()
            bed_number = int(bed_number_str.replace('bed', ''))
            complete_data = {
                "bed": bed_number,
                "temperature": vital_signs_data["temperature"],
                "heart_rate": vital_signs_data["heart_rate"],
                "spo2": vital_signs_data["spo2"],
                "breath_rate": vital_signs_data["breath_rate"],
                "blood_pressure": blood_pressure_value,
                "timestamp": datetime.datetime.now()
            }
            complete_vital_signs_data.append(complete_data)
            try:
                val_heart_rate = float(vital_signs_data["heart_rate"])
                val_spo2 = float(vital_signs_data["spo2"])
                val_breath_rate = float(vital_signs_data["breath_rate"])
                val_temperature = float(vital_signs_data["temperature"])
                health_prediction, rules = predict_health_state(val_heart_rate, blood_pressure_value, val_spo2, val_breath_rate, val_temperature)
                db = mongo_client["hopital"]

                
                for signal, value in vital_signs_data.items():
                    collection = db[signal]
                    data = {
                        "bed": bed_number,
                        "value": value,
                        "timestamp": datetime.datetime.now()
                    }
                    collection.insert_one(data)
                    print(f"Data inserted into collection '{signal}' in database 'hopital'")
                collection = db["etat_patient"]
                patient_data = {
                    "bed": bed_number,
                    "health_prediction": health_prediction,
                    "rules": rules,
                    "timestamp": datetime.datetime.now()
                }
                collection.insert_one(patient_data)
                print("Data inserted into collection 'etat_patient' in database 'hopital'")
                if not (60 <= val_heart_rate <= 100) or \
                   not (100 <= blood_pressure_value <= 140) or \
                   not (val_spo2 > 92) or \
                   not (12 <= val_breath_rate <= 20) or \
                   not (36 <= val_temperature <= 38):
                    phone_number=get_today_phone_numbers()
                    message = f"anomalie dans le lit : {bed_number}"
                    send_order(phone_number[0], message)
                    print(f"********un message durgence est envoyer lit numero",bed_number,"et numero de tel",phone_number)




                for key in vital_signs_data:
                    vital_signs_data[key] = None
            except ValueError as e:
                print(f"Erreur de conversion de la valeur: {e}")
                print(traceback.format_exc())

    elif signal_type == "ecg":
        try:
            ecg_data = json.loads(msg.payload.decode())
            if isinstance(ecg_data, list):
                ecg_values.extend(ecg_data)
                if len(ecg_values) >= 3600:
                    ecg_signal = np.array(ecg_values[:3600])
                    scaler = StandardScaler()
                    ecg_signal_standardized = scaler.fit_transform(ecg_signal.reshape(-1, 1)).flatten()
                    signal_features = extract_features(ecg_signal_standardized)
                    dtest = xgb.DMatrix(signal_features.reshape(1, -1))
                    predicted_class = int(xgboost_model.predict(dtest)[0])
                    data = {
                        "bed": int(bed_number_str.replace('bed', '')),
                        "values": ecg_values[:3600],
                        "valeur_predite": predicted_class,
                        "timestamp": datetime.datetime.now()
                    }
                    db = mongo_client["hopital"]
                    collection = db["ecg"]
                    collection.insert_one(data)
                    print(f"Data inserted into collection 'ecg' in database 'hopital'")
                    ecg_values[:] = ecg_values[3600:]
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON message: {e}")

# Configuration du client MQTT
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_broker, mqtt_port, 60)

# Boucle infinie pour maintenir la connexion MQTT
client.loop_forever()


c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\dell\AppData\Local\Temp\ipykernel_11744\1579121125.py:251: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with fea

Connected to MQTT broker
Received `29.85` from `vital_signs/bed1/temperature` topic
Received `-999` from `vital_signs/bed1/heart_rate` topic
Received `-999` from `vital_signs/bed1/spo2` topic
Received `11` from `vital_signs/bed1/breath_rate` topic
État de santé prédit: Pas Normal
Règles pour l'échantillon 0:
Decision node 0: (X[respiratory_rate] <= 18.50)
Decision node 1: (X[blood_pressure] <= 121.00)
Decision node 2: (X[blood_pressure] > 89.50)
Decision node 4: (X[respiratory_rate] > 8.50)
Decision node 6: (X[temperature] <= 38.05)
Decision node 7: (X[heart_rate] <= 101.50)
Decision node 8: (X[temperature] <= 35.45)
Data inserted into collection 'temperature' in database 'hopital'
Data inserted into collection 'heart_rate' in database 'hopital'
Data inserted into collection 'spo2' in database 'hopital'
Data inserted into collection 'breath_rate' in database 'hopital'
Data inserted into collection 'etat_patient' in database 'hopital'
Sending order: 0670012189;anomalie dans le lit : 1



c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Data inserted into collection 'heart_rate' in database 'hopital'
Data inserted into collection 'spo2' in database 'hopital'
Data inserted into collection 'breath_rate' in database 'hopital'
Data inserted into collection 'etat_patient' in database 'hopital'
Sending order: 0670012189;anomalie dans le lit : 1

********un message durgence est envoyer lit numero 1 et numero de tel ['0670012189']
Received `[1729,1759,1834,1897,1841,1818,1770,1741,1753,1822,1882,1838,1819,1778,1741,1761,1815,1871,1821,1806,1776,1743,1782,1846,1877,1840,1815,1779,1744,1779,1849,1883,1858,1818,1776,1742,1776,1842,1863,1847,1823,1783,1745,1775,1840,1859,1842,1811,1767,1735,1776,1833,1853,1845,1827,1794,1750,1775,1839,1866,1855,1826,1777,1739,1771,1822,1845,1841,1808,1776,1746,1780,1840,1842,1847,1811,1774,1734,1775,1835,1853,1847,1814,1775,1744,1779,1840,1857,1840,1806,1776,1746,1790,1850,1872,1826,1783,1747,1747,1789,1861,1870,1857,1817,1776,1744,1778,1850,1875,1849,1808,1776,1754,1798,1854,1885,1853,1801,1776,

c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


...

c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Sending order: 0670012189;anomalie dans le lit : 1

********un message durgence est envoyer lit numero 1 et numero de tel ['0670012189']
Received `[4095,4095,4095,4095,3040,971,3826,4095,4095,4095,4095,2719,479,3828,4095,4095,4095,4095,2533,398,3987,4095,4095,4095,4095,2481,416,4095,4095,4095,4095,4095,2434,490,4095,4095,4095,4095,4095,2240,93,3877,4095,4095,4095,4095,2058,0,3437,4095,4095,4095,4095,1905,0,3630,4095,4095,4095,4095,1675,0,2898,4095,4095,4095,4095,1200,0,2673,4095,4095,4095,4095,976,0,2739,4095,4095,4095,4095,1206,0,3163,4095,4095,4095,4095,1392,0,3179,4095,4095,4095,4095,1094,0,2725,4095,4095,4095,4095,896,0,3319,4095,4095,4095,4095,928,0,2959,4095,4095,4095,4095,741,0,3519,4095,4095,4095,4095,1472,432,4095,4095,4095,4095,4095,1121,0,4095,4095,4095,4095,4095,967,0,3470,4095,4095,4095,4095,678,0,4095,4095,4095,4095,4095,1137,0,4095,4095,4095,4095,4095,1023,0,4095,4095,4095,4095,4095,1329,373,4095,4095,4095,4095,4095,1172,256,4095,4095,4095,4095,4095,1106,207,4095,4095,40

c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Data inserted into collection 'heart_rate' in database 'hopital'
Data inserted into collection 'spo2' in database 'hopital'
Data inserted into collection 'breath_rate' in database 'hopital'
Data inserted into collection 'etat_patient' in database 'hopital'
Sending order: 0670012189;anomalie dans le lit : 1

********un message durgence est envoyer lit numero 1 et numero de tel ['0670012189']
Received `[4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,4095,

ValueError: Not enough beats to compute heart rate.